In [3]:
import cv2
import os, glob
import numpy as np
from PIL import Image
import torch as th
import torchvision

# open the video file
# vid_name = './videos/Anakin_Trim_2.mp4'

def sort_by_frame(path_list):
    frame_anno = []
    for p in path_list:
        # frame_idx = os.path.splitext(p.split('_')[-1])[0][5:]   # 0-4 is "frame", so we used [5:] here
        frame_idx = os.path.splitext(p.split('/')[-1].split('_')[-1])[0][5:]   # 0-4 is "frame", so we used [5:] here
        frame_anno.append(int(frame_idx))
    sorted_idx = np.argsort(frame_anno)
    sorted_path_list = []
    for idx in sorted_idx:
      sorted_path_list.append(path_list[idx])
    return sorted_path_list

track_name = 'Denarys_1'
track_path = f"/data/mint/sampling/Videos/log=Masked_Face_woclip+BgNoHead+shadow_256_cfg=Masked_Face_woclip+BgNoHead+shadow_256.yaml/ema_085000/valid/reverse_sampling/src={track_name}"
assert os.path.exists(track_path)

avail_light = os.listdir(f'/{track_path}/')
print("Avaliable_light : ", avail_light)
light_file = avail_light[int(input("Choose light : "))]
relit_path = f"{track_path}/{light_file}/diff=1000/"
src_path = f"/data/mint/DPM_Dataset/Videos/{track_name}/aligned_images/valid/"
all_type = []
for f_type in ['ren_f', 'ren_relit_f', 'res_f']:
    fl = glob.glob(f'/{relit_path}/{f_type}*')
    fl = sort_by_frame(fl)
    if f_type == 'res_f':
        src_f = [f"/{src_path}/{f.split('/')[-1].split('.')[0].split('_')[-1]}.png" for f in fl]
        src_frames = th.stack([th.tensor(np.array(Image.open(f))) for f in src_f], dim=0)
        all_type.append(src_frames)
        
    frames = th.stack([th.tensor(np.array(Image.open(f))) for f in fl], dim=0)
    
    all_type.append(frames)
    torchvision.io.write_video(video_array=frames, filename=f'./{f_type}.mp4', fps=20)
    
all_frames_type = th.cat(all_type, dim=2)
torchvision.io.write_video(video_array=all_frames_type, filename=f'./out_{light_file}.mp4', fps=20)

Avaliable_light :  ['light=63502', 'light=60623_light', 'light=Denarys2_light']
